<a href="https://colab.research.google.com/github/ptk18/ADA_dynamic_programming/blob/master/Mm_mT5_Vocab_Pruning_Training_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets huggingface_hub accelerate -q

print("Packages installed successfully!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 32.1 MB/s eta 0:00:00
Packages installed successfully!


In [ ]:
import torch
from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq
)
from datasets import load_dataset, Dataset, DatasetDict
from huggingface_hub import login, HfApi
import pandas as pd
from collections import Counter
import numpy as np
from tqdm.auto import tqdm

print("Libraries imported successfully!")
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")

Libraries imported successfully!
GPU available: True
GPU name: Tesla T4


In [ ]:
# Login to HuggingFace
login()

username = 'ptk18'
print(f"Username set to: {username}")

Username set to: ptk18


In [ ]:
# Load Myanmar news dataset
print("Loading Myanmar News Dataset...")

try:
    from huggingface_hub import hf_hub_download

    print("Downloading train data...")
    train_file = hf_hub_download(
        repo_id="ThuraAung1601/myanmar_news",
        filename="data/train-00000-of-00001.parquet",
        repo_type="dataset"
    )

    print("Downloading test data...")
    test_file = hf_hub_download(
        repo_id="ThuraAung1601/myanmar_news",
        filename="data/test-00000-of-00001.parquet",
        repo_type="dataset"
    )

    # Load with pandas
    train_df = pd.read_parquet(train_file)
    test_df = pd.read_parquet(test_file)

    # Convert to datasets
    dataset = DatasetDict({
        'train': Dataset.from_pandas(train_df),
        'test': Dataset.from_pandas(test_df)
    })

    print("SUCCESS! Myanmar dataset loaded")
    print(f"Train samples: {len(dataset['train'])}")
    print(f"Test samples: {len(dataset['test'])}")

    # Show sample
    sample = dataset['train'][0]
    print(f"\nSample text: {sample['text'][:100]}...")
    print(f"Sample label: {sample['label']}")

except Exception as e:
    print(f"Failed to load dataset: {e}")
    print("Please check if you have access to the dataset")

Loading Myanmar News Dataset...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


train-00000-of-00001.parquet:   0%|          | 0.00/1.54M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/391k [00:00<?, ?B/s]

SUCCESS! Myanmar dataset loaded
Train samples: 5844
Test samples: 1471

Sample text: ၂၀၂၃ ၂၀၂၄ ဘဏ္ဍာ နှစ် ငါး လ အတွင်း ပြည်ပ သို့ ရေ ထွက် ပစ္စည်း တင် ပို့ မှု မှ ဒေါ်လာ ၂၄၁ သန်း ကျော် ရ...
Sample label: __label__Business


In [ ]:
# Load original mT5 model
print("Loading original mT5-small model...")

model_name = "google/mt5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

print("Original model loaded successfully!")
print(f"Original model parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Original vocabulary size: {tokenizer.vocab_size:,}")

# Check model size breakdown
total_params = sum(p.numel() for p in model.parameters())
shared_params = sum(p.numel() for p in model.shared.parameters())
lm_head_params = sum(p.numel() for p in model.lm_head.parameters())

print(f"\nModel breakdown:")
print(f"Total parameters: {total_params:,}")
print(f"Shared embeddings: {shared_params:,} ({shared_params/total_params*100:.1f}%)")
print(f"LM head: {lm_head_params:,} ({lm_head_params/total_params*100:.1f}%)")
print(f"Other: {total_params-shared_params-lm_head_params:,}")

Loading original mT5-small model...


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Original model loaded successfully!
Original model parameters: 300,176,768
Original vocabulary size: 250,100

Model breakdown:
Total parameters: 300,176,768
Shared embeddings: 128,057,344 (42.7%)
LM head: 128,057,344 (42.7%)
Other: 44,062,080


In [ ]:
# Analyze Myanmar token usage
print("Analyzing Myanmar token usage...")

# Collect Myanmar texts
all_myanmar_texts = []

# Add original dataset texts
for i, sample in enumerate(dataset['train']):
    if i >= 1000:  # Analyze first 1000 samples
        break
    all_myanmar_texts.append(sample['text'])

# Add classification format texts
classification_texts = []
for i, sample in enumerate(dataset['train']):
    if i >= 200:  # 200 samples for classification format
        break
    classification_text = f"classify: {sample['text']}"
    classification_texts.append(classification_text)

all_texts_to_analyze = all_myanmar_texts + classification_texts

print(f"Analyzing {len(all_texts_to_analyze)} Myanmar texts...")

# Count token frequencies
myanmar_token_counter = Counter()

for text in tqdm(all_texts_to_analyze):
    tokens = tokenizer.encode(text, add_special_tokens=False)
    myanmar_token_counter.update(tokens)

print(f"Analysis complete!")
print(f"Unique Myanmar tokens found: {len(myanmar_token_counter):,}")
print(f"Percentage of vocab used: {len(myanmar_token_counter)/tokenizer.vocab_size*100:.1f}%")

# Show token coverage
total_myanmar_tokens = sum(myanmar_token_counter.values())
cumulative_freq = 0

print(f"\nToken coverage analysis:")
for threshold in [1000, 5000, 10000, 15000, 20000]:
    if threshold <= len(myanmar_token_counter):
        cumulative_freq = sum(freq for _, freq in myanmar_token_counter.most_common(threshold))
        coverage = cumulative_freq / total_myanmar_tokens * 100
        print(f"Top {threshold:,} tokens cover {coverage:.2f}% of Myanmar text")

Analyzing Myanmar token usage...
Analyzing 1200 Myanmar texts...


  0%|          | 0/1200 [00:00<?, ?it/s]

Analysis complete!
Unique Myanmar tokens found: 1,178
Percentage of vocab used: 0.5%

Token coverage analysis:
Top 1,000 tokens cover 99.63% of Myanmar text


In [ ]:
#Create pruned vocabulary
print("Creating pruned vocabulary...")

new_tokens = set()

# Safety tokens (top 1000 from original vocab)
safety_tokens = 1000
new_tokens.update(range(safety_tokens))
print(f"Added {safety_tokens:,} safety tokens")

# Myanmar tokens (most frequent)
myanmar_tokens_target = 15000
myanmar_tokens_added = 0

for token_id, freq in myanmar_token_counter.most_common():
    if myanmar_tokens_added >= myanmar_tokens_target:
        break
    if token_id not in new_tokens:
        new_tokens.add(token_id)
        myanmar_tokens_added += 1

print(f"Added {myanmar_tokens_added:,} Myanmar tokens")

# English tokens (approximate common range)
english_start = 10
english_end = 8000
english_tokens_added = 0

for token_id in range(english_start, english_end):
    if len(new_tokens) >= 24900:  # Leave room for special tokens
        break
    if token_id not in new_tokens:
        new_tokens.add(token_id)
        english_tokens_added += 1

print(f"Added {english_tokens_added:,} English tokens")

# Special tokens (T5 control tokens)
special_tokens = 100
for token_id in range(tokenizer.vocab_size - special_tokens, tokenizer.vocab_size):
    new_tokens.add(token_id)

print(f"Added {special_tokens} special tokens")

# Final vocabulary
kept_ids = sorted(list(new_tokens))
new_vocab_size = len(kept_ids)

print(f"\nVocabulary pruning plan:")
print(f"Original vocabulary: {tokenizer.vocab_size:,} tokens")
print(f"New vocabulary: {new_vocab_size:,} tokens")
print(f"Reduction: {(1 - new_vocab_size/tokenizer.vocab_size)*100:.1f}%")

Creating pruned vocabulary...
Added 1,000 safety tokens
Added 1,175 Myanmar tokens
Added 6,984 English tokens
Added 100 special tokens

Vocabulary pruning plan:
Original vocabulary: 250,100 tokens
New vocabulary: 9,259 tokens
Reduction: 96.3%


In [ ]:
# Create pruned model
print("Creating pruned model...")

# Create token mapping
old_to_new = {old_id: new_id for new_id, old_id in enumerate(kept_ids)}
new_to_old = {new_id: old_id for new_id, old_id in enumerate(kept_ids)}

print(f"Token mapping created: {len(old_to_new):,} mappings")

# Create new embeddings with reduced vocabulary
print("Creating new embeddings...")
new_embedding = torch.nn.Embedding(new_vocab_size, model.shared.embedding_dim)
new_lm_head = torch.nn.Linear(
    in_features=model.lm_head.in_features,
    out_features=new_vocab_size,
    bias=False
)

# Copy weights for kept tokens
print("Copying weights for kept tokens...")
for new_id, old_id in enumerate(tqdm(kept_ids)):
    new_embedding.weight.data[new_id] = model.shared.weight.data[old_id]
    new_lm_head.weight.data[new_id] = model.lm_head.weight.data[old_id]

# Update model
model.shared = new_embedding
model.lm_head = new_lm_head
model.config.vocab_size = new_vocab_size

print("Model pruning completed!")

# Check new model size
new_total_params = sum(p.numel() for p in model.parameters())
reduction_percentage = (1 - new_total_params/60506624) * 100  # Original mT5-small size

print(f"\nPruned model statistics:")
print(f"New model parameters: {new_total_params:,}")
print(f"Original model parameters: 60,506,624")
print(f"Size reduction: {reduction_percentage:.1f}%")

# Move to GPU
model = model.cuda()
print("Pruned model moved to GPU")

Creating pruned model...
Token mapping created: 9,259 mappings
Creating new embeddings...
Copying weights for kept tokens...


  0%|          | 0/9259 [00:00<?, ?it/s]

Model pruning completed!

Pruned model statistics:
New model parameters: 181,600,640
Original model parameters: 60,506,624
Size reduction: -200.1%
Pruned model moved to GPU


In [ ]:
# Create custom tokenization for pruned model
print("Creating custom tokenization function...")

def tokenize_with_pruned_vocab(text, max_length=256):
    # Tokenize with original tokenizer
    original_tokens = tokenizer.encode(text, add_special_tokens=True, max_length=max_length, truncation=True)

    # Map to new vocabulary
    mapped_tokens = []
    for token_id in original_tokens:
        if token_id in old_to_new:
            mapped_tokens.append(old_to_new[token_id])
        else:
            # Map unknown tokens to UNK (assume token 0 exists in new vocab)
            mapped_tokens.append(old_to_new.get(0, 0))

    return mapped_tokens

def batch_tokenize_with_pruned_vocab(texts, max_length=256):
    """Batch tokenize for training"""
    tokenized = []
    for text in texts:
        tokens = tokenize_with_pruned_vocab(text, max_length)
        tokenized.append(tokens)
    return tokenized

# Test the tokenization
test_text = "classify: နည်းပညာ ကဏ္ဍတွင် အသစ်သော တီထွင်မှုများ"
test_tokens = tokenize_with_pruned_vocab(test_text)

print(f"Test tokenization:")
print(f"Text: {test_text[:50]}...")
print(f"Tokens: {len(test_tokens)} tokens")
print(f"First 10 tokens: {test_tokens[:10]}")

print("Custom tokenization ready!")

Creating custom tokenization function...
Test tokenization:
Text: classify: နည်းပညာ ကဏ္ဍတွင် အသစ်သော တီထွင်မှုများ...
Tokens: 18 tokens
First 10 tokens: [3931, 8076, 267, 259, 8254, 8370, 259, 9150, 8046, 259]
Custom tokenization ready!


In [ ]:
# Prepare training data with custom tokenization
print("Preparing training data...")

def prepare_classification_data(dataset_split, max_samples=1000):
    inputs = []
    targets = []

    for i, sample in enumerate(dataset_split):
        if i >= max_samples:
            break

        # Create T5 format
        input_text = f"classify: {sample['text']}"
        target_text = sample['label'].replace("__label__", "")

        inputs.append(input_text)
        targets.append(target_text)

    return inputs, targets

# Prepare data
train_inputs, train_targets = prepare_classification_data(dataset['train'], max_samples=1000)
test_inputs, test_targets = prepare_classification_data(dataset['test'], max_samples=200)

print(f"Training samples: {len(train_inputs)}")
print(f"Test samples: {len(test_inputs)}")

# Show sample
print(f"\nSample input: {train_inputs[0][:60]}...")
print(f"Sample target: {train_targets[0]}")

# Tokenize with pruned vocabulary
print("Tokenizing with pruned vocabulary...")

train_input_tokens = batch_tokenize_with_pruned_vocab(train_inputs, max_length=256)
train_target_tokens = batch_tokenize_with_pruned_vocab(train_targets, max_length=32)
test_input_tokens = batch_tokenize_with_pruned_vocab(test_inputs, max_length=256)
test_target_tokens = batch_tokenize_with_pruned_vocab(test_targets, max_length=32)

print("Tokenization completed!")

# Create datasets
train_dataset = Dataset.from_dict({
    'input_ids': train_input_tokens,
    'labels': train_target_tokens
})

test_dataset = Dataset.from_dict({
    'input_ids': test_input_tokens,
    'labels': test_target_tokens
})

print(f"Datasets created:")
print(f"Train dataset: {len(train_dataset)} samples")
print(f"Test dataset: {len(test_dataset)} samples")

Preparing training data...
Training samples: 1000
Test samples: 200

Sample input: classify: ၂၀၂၃ ၂၀၂၄ ဘဏ္ဍာ နှစ် ငါး လ အတွင်း ပြည်ပ သို့ ရေ ထွ...
Sample target: Business
Tokenizing with pruned vocabulary...
Tokenization completed!
Datasets created:
Train dataset: 1000 samples
Test dataset: 200 samples


In [ ]:
# Setup training arguments
print("Setting up training configuration...")

# Model name for saving
model_name_hf = f"{username}/myanmar-t5-vocab-pruned"
print(f"Model will be saved as: {model_name_hf}")

# Training arguments
training_args = TrainingArguments(
    output_dir="./myanmar-t5-vocab-pruned",

    num_train_epochs=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,

    learning_rate=5e-5,
    warmup_steps=50,
    weight_decay=0.01,

    eval_strategy="steps",
    eval_steps=50,
    logging_steps=25,

    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,

    push_to_hub=False,
    hub_model_id=model_name_hf,

    fp16=True if torch.cuda.is_available() else False,
    dataloader_num_workers=2,
    load_best_model_at_end=True,

    report_to=None,
)

print("Training configuration ready!")
print(f"Training details:")
print(f"- Myanmar samples: {len(train_dataset)}")
print(f"- Test samples: {len(test_dataset)}")
print(f"- Epochs: 2")
print(f"- Pruned vocabulary: {new_vocab_size:,} tokens")
print(f"- Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")

Setting up training configuration...
Model will be saved as: ptk18/myanmar-t5-vocab-pruned
Training configuration ready!
Training details:
- Myanmar samples: 1000
- Test samples: 200
- Epochs: 2
- Pruned vocabulary: 9,259 tokens
- Device: GPU


In [ ]:
# Create custom data collator for pruned vocabulary
print("Creating custom data collator...")

class PrunedVocabDataCollator:
    """Custom data collator for pruned vocabulary model"""

    def __init__(self, pad_token_id=0, max_length=256):
        self.pad_token_id = pad_token_id
        self.max_length = max_length

    def __call__(self, features):
        # Pad input_ids
        max_input_len = max(len(f['input_ids']) for f in features)
        max_input_len = min(max_input_len, self.max_length)

        # Pad labels
        max_label_len = max(len(f['labels']) for f in features)
        max_label_len = min(max_label_len, 64)  # Shorter for labels

        input_ids = []
        attention_mask = []
        labels = []

        for feature in features:
            # Pad input_ids
            input_id = feature['input_ids'][:max_input_len]
            pad_len = max_input_len - len(input_id)
            input_id = input_id + [self.pad_token_id] * pad_len
            input_ids.append(input_id)

            # Create attention mask
            mask = [1] * len(feature['input_ids'][:max_input_len]) + [0] * pad_len
            attention_mask.append(mask)

            # Pad labels (use -100 for padding)
            label = feature['labels'][:max_label_len]
            label_pad_len = max_label_len - len(label)
            label = label + [-100] * label_pad_len
            labels.append(label)

        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
            'labels': torch.tensor(labels, dtype=torch.long)
        }

# Create the custom data collator
data_collator = PrunedVocabDataCollator(pad_token_id=old_to_new.get(0, 0))

print("Custom data collator created!")

# Test the data collator
print("Testing data collator...")
sample_batch = [train_dataset[i] for i in range(3)]
collated = data_collator(sample_batch)

print("Collated batch structure:")
for key, value in collated.items():
    print(f"  {key}: shape {value.shape}")

print("Data collator working correctly!")

Creating custom data collator...
Custom data collator created!
Testing data collator...
Collated batch structure:
  input_ids: shape torch.Size([3, 118])
  attention_mask: shape torch.Size([3, 118])
  labels: shape torch.Size([3, 2])
Data collator working correctly!


In [ ]:
# Create trainer and start training
print("Creating trainer...")

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
)

print("Trainer created successfully!")
print("Starting training on Myanmar news with pruned vocabulary...")
print("This should take 15-30 minutes...")

# Start training
try:
    training_output = trainer.train()

    print("Training completed successfully!")
    print(f"Final training loss: {training_output.training_loss:.4f}")

    # Check if we got reasonable loss values
    if training_output.training_loss > 0 and training_output.training_loss < 10:
        print("Training loss looks good - model learned!")
    else:
        print("Training loss looks unusual - check results")

except Exception as e:
    print(f"Training failed: {e}")
    print("Check the error and data format")

Creating trainer...


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Trainer created successfully!
Starting training on Myanmar news with pruned vocabulary...
This should take 15-30 minutes...


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ptk18 (ptk18-king-mongkut-s-institute-of-technology-ladkrabang) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
50,0.000000,nan
100,0.000000,nan
150,0.000000,nan
200,0.000000,nan
250,0.000000,nan


Training completed successfully!
Final training loss: 0.0000
Training loss looks unusual - check results


In [ ]:
# Test if the model actually learned anything
print("Testing the trained model...")

test_cases = [
    "classify: နည်းပညာ ကဏ္ဍတွင် အသစ်သော တီထွင်မှုများ ပေါ်ပေါက်လာခဲ့သည်။",
    "classify: ဘောလုံး ပြိုင်ပွဲတွင် မြန်မာအသင်း အနိုင်ရရှိခဲ့သည်။",
    "classify: စီးပွားရေး အခြေအနေ တိုးတက်လာခဲ့ပါတယ်။",
]

expected = ["Science&Technology", "Sports", "Business"]

print("Model test results:")
for i, (test_text, exp) in enumerate(zip(test_cases, expected)):
    # Use our custom tokenization
    test_tokens = tokenize_with_pruned_vocab(test_text, max_length=128)
    inputs = torch.tensor([test_tokens])

    if torch.cuda.is_available():
        inputs = inputs.cuda()

    try:
        with torch.no_grad():
            outputs = model.generate(
                inputs,
                max_length=50,
                num_beams=2,
                early_stopping=True
            )

        # Decode with mapping back to original tokens
        result_tokens = outputs[0].cpu().tolist()

        # Try to decode (this might not work perfectly due to vocab pruning)
        print(f"\nTest {i+1}:")
        print(f"Input: {test_text[:50]}...")
        print(f"Expected: {exp}")
        print(f"Generated token IDs: {result_tokens[:10]}")

        # Check if output looks reasonable
        if len(set(result_tokens)) > 1:  # Not all same tokens
            print("Status: Model generated different tokens")
        else:
            print("Status: Model outputs same token repeatedly")

    except Exception as e:
        print(f"Test {i+1} failed: {e}")

Testing the trained model...
Model test results:

Test 1:
Input: classify: နည်းပညာ ကဏ္ဍတွင် အသစ်သော တီထွင်မှုများ ပ...
Expected: Science&Technology
Generated token IDs: [0, 0]
Status: Model outputs same token repeatedly

Test 2:
Input: classify: ဘောလုံး ပြိုင်ပွဲတွင် မြန်မာအသင်း အနိုင်...
Expected: Sports
Generated token IDs: [0, 0]
Status: Model outputs same token repeatedly

Test 3:
Input: classify: စီးပွားရေး အခြေအနေ တိုးတက်လာခဲ့ပါတယ်။...
Expected: Business
Generated token IDs: [0, 0]
Status: Model outputs same token repeatedly


In [ ]:
# Try simple training without vocabulary pruning
print("Trying simple approach without vocabulary pruning...")

# Clear memory completely
import torch
import gc
torch.cuda.empty_cache()
gc.collect()

# Load completely fresh, unmodified model
print("Loading fresh, unmodified mT5...")
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_name = "google/mt5-small"
tokenizer_fresh = T5Tokenizer.from_pretrained(model_name)
model_fresh = T5ForConditionalGeneration.from_pretrained(model_name)

if torch.cuda.is_available():
    model_fresh = model_fresh.cuda()

print("Fresh model loaded - no modifications")
print(f"Vocabulary size: {tokenizer_fresh.vocab_size:,}")
print(f"Model parameters: {sum(p.numel() for p in model_fresh.parameters()):,}")

# Test the fresh model BEFORE any training
print("\nTesting fresh model before training...")
test_input = "translate English to German: Hello world"
inputs = tokenizer_fresh(test_input, return_tensors="pt")
if torch.cuda.is_available():
    inputs = {k: v.cuda() for k, v in inputs.items()}

with torch.no_grad():
    outputs = model_fresh.generate(**inputs, max_length=20)

fresh_result = tokenizer_fresh.decode(outputs[0], skip_special_tokens=True)
print(f"Fresh model test: {fresh_result}")

if len(fresh_result) > 5 and fresh_result != test_input:
    print("✓ Fresh model works correctly")
else:
    print("✗ Fresh model has issues")

Trying simple approach without vocabulary pruning...
Loading fresh, unmodified mT5...


You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


Fresh model loaded - no modifications
Vocabulary size: 250,100
Model parameters: 300,176,768

Testing fresh model before training...
Fresh model test: <extra_id_0>
✓ Fresh model works correctly


In [ ]:
# Simple Myanmar training with minimal changes
print("Setting up simple Myanmar training...")

# Use very small dataset and simple format
simple_myanmar_inputs = [
    "classify: နည်းပညာ သတင်း",
    "classify: အားကစား သတင်း",
    "classify: စီးပွားရေး သတင်း",
    "classify: နိုင်ငံရေး သတင်း",
    "classify: ဖျော်ဖြေရေး သတင်း",
] * 20  # 100 samples

simple_myanmar_targets = [
    "Technology",
    "Sports",
    "Business",
    "Politics",
    "Entertainment",
] * 20

print(f"Simple dataset: {len(simple_myanmar_inputs)} samples")

# Simple tokenization with original tokenizer
def tokenize_simple_myanmar(examples):
    # Tokenize inputs
    model_inputs = tokenizer_fresh(
        examples['input_text'],
        max_length=64,  # Very short
        truncation=True,
        padding=False
    )

    # Tokenize targets
    with tokenizer_fresh.as_target_tokenizer():
        labels = tokenizer_fresh(
            examples['target_text'],
            max_length=16,  # Very short labels
            truncation=True,
            padding=False
        )

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Create simple dataset
simple_dataset = Dataset.from_dict({
    'input_text': simple_myanmar_inputs,
    'target_text': simple_myanmar_targets
})

simple_tokenized = simple_dataset.map(tokenize_simple_myanmar, batched=True)
simple_tokenized = simple_tokenized.remove_columns(['input_text', 'target_text'])

print("Simple dataset tokenized")

# Check a sample
sample = simple_tokenized[0]
print(f"Sample input: {tokenizer_fresh.decode(sample['input_ids'])}")
print(f"Sample label: {tokenizer_fresh.decode(sample['labels'])}")

Setting up simple Myanmar training...
Simple dataset: 100 samples


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Simple dataset tokenized
Sample input: classify: နည်းပညာ သတင်း</s>
Sample label: Technology</s>


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3951: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
# Minimal training test
print("Starting minimal training test...")

# Ultra-minimal training arguments (CORRECTED)
training_args_minimal = TrainingArguments(
    output_dir="./minimal-myanmar",
    max_steps=20,  # Only 20 steps
    per_device_train_batch_size=2,
    learning_rate=5e-5,
    logging_steps=5,
    save_strategy="no",
    eval_strategy="no",  # CORRECTED: was evaluation_strategy
    push_to_hub=False,
    report_to=None,
    fp16=torch.cuda.is_available(),
    dataloader_num_workers=0,
)

# Standard data collator
data_collator_standard = DataCollatorForSeq2Seq(
    tokenizer=tokenizer_fresh,
    model=model_fresh,
    padding=True
)

# Create trainer with minimal setup
trainer_minimal = Trainer(
    model=model_fresh,
    args=training_args_minimal,
    train_dataset=simple_tokenized.select(range(20)),  # Only 20 samples
    data_collator=data_collator_standard,
)

print("Starting 20-step minimal training...")
try:
    training_result = trainer_minimal.train()

    print(f"Minimal training completed!")
    print(f"Final loss: {training_result.training_loss:.6f}")

    if training_result.training_loss > 0:
        print("SUCCESS: Training loss is positive - learning happened!")

        # Test the minimally trained model
        print("\nTesting minimally trained model...")
        test_input = "classify: နည်းပညာ သတင်း"
        inputs = tokenizer_fresh(test_input, return_tensors="pt")
        if torch.cuda.is_available():
            inputs = {k: v.cuda() for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model_fresh.generate(
                **inputs,
                max_length=30,
                num_beams=2,
                early_stopping=True
            )

        result = tokenizer_fresh.decode(outputs[0], skip_special_tokens=True)
        print(f"Result: {result}")

        if "Technology" in result:
            print("EXCELLENT: Model learned to classify!")
        elif len(result) > 10 and result != test_input:
            print("GOOD: Model generated something different")
        else:
            print("Model still has issues but training worked")

    else:
        print("FAILED: Training loss is 0 - no learning happened")

except Exception as e:
    print(f"Minimal training failed: {e}")
    print("Even basic training is not working")

Starting minimal training test...
Starting 20-step minimal training...


Step,Training Loss
5,0.000000
10,0.000000
15,0.000000
20,0.000000


Minimal training completed!
Final loss: 0.000000
FAILED: Training loss is 0 - no learning happened


In [ ]:
# Training with 1 epoch and debugging
print("Trying 1 epoch with debugging...")

# Check the training data first
print("Debugging training data...")
sample_batch = [simple_tokenized[i] for i in range(3)]
collated_batch = data_collator_standard(sample_batch)

print("Batch structure:")
for key, value in collated_batch.items():
    if torch.is_tensor(value):
        print(f"  {key}: shape {value.shape}, sample values {value[0][:5]}")

# Test manual forward pass
print("\nTesting manual forward pass...")
if torch.cuda.is_available():
    batch_cuda = {k: v.cuda() if torch.is_tensor(v) else v for k, v in collated_batch.items()}
else:
    batch_cuda = collated_batch

model_fresh.train()
try:
    outputs = model_fresh(**batch_cuda)
    loss = outputs.loss
    print(f"Manual forward pass loss: {loss.item()}")

    if loss.item() == 0.0:
        print("PROBLEM: Loss is exactly 0 in forward pass!")
        print("This means model/data setup is wrong")
    else:
        print("Forward pass working - loss is normal")

except Exception as e:
    print(f"Manual forward pass failed: {e}")

# Try 1 epoch training
print("\nTrying 1 epoch training...")
training_args_1epoch = TrainingArguments(
    output_dir="./myanmar-1epoch",
    num_train_epochs=1,  # Only 1 epoch
    per_device_train_batch_size=1,  # Batch size 1
    learning_rate=1e-4,  # Higher learning rate
    logging_steps=2,
    save_strategy="no",
    eval_strategy="no",
    push_to_hub=False,
    report_to=None,
    fp16=False,  # Disable fp16
    dataloader_num_workers=0,
)

trainer_1epoch = Trainer(
    model=model_fresh,
    args=training_args_1epoch,
    train_dataset=simple_tokenized.select(range(10)),  # Only 10 samples
    data_collator=data_collator_standard,
)

try:
    result_1epoch = trainer_1epoch.train()
    print(f"1 epoch result: loss = {result_1epoch.training_loss}")

    if result_1epoch.training_loss > 0:
        print("SUCCESS with 1 epoch!")
    else:
        print("Still failed with 1 epoch")

except Exception as e:
    print(f"1 epoch training failed: {e}")

Trying 1 epoch with debugging...
Debugging training data...
Batch structure:
  input_ids: shape torch.Size([3, 8]), sample values tensor([ 3931, 30077,   267,   259, 80518])
  attention_mask: shape torch.Size([3, 8]), sample values tensor([1, 1, 1, 1, 1])
  labels: shape torch.Size([3, 2]), sample values tensor([11947,     1])
  decoder_input_ids: shape torch.Size([3, 2]), sample values tensor([    0, 11947])

Testing manual forward pass...
Manual forward pass loss: nan
Forward pass working - loss is normal

Trying 1 epoch training...


Step,Training Loss
2,0.000000
4,0.000000
6,0.000000
8,0.000000
10,0.000000


1 epoch result: loss = 0.0
Still failed with 1 epoch


In [ ]:
# Debug NaN loss issue
print("Debugging NaN loss...")

# Check vocabulary size vs token IDs
print(f"Model vocabulary size: {model_fresh.config.vocab_size}")
print(f"Tokenizer vocabulary size: {tokenizer_fresh.vocab_size}")

# Check the problematic batch
sample_batch = [simple_tokenized[i] for i in range(2)]  # Just 2 samples
collated = data_collator_standard(sample_batch)

print("Checking token ID ranges...")
input_ids = collated['input_ids']
labels = collated['labels']

print(f"Input IDs range: min={input_ids.min()}, max={input_ids.max()}")
print(f"Labels range: min={labels[labels != -100].min()}, max={labels[labels != -100].max()}")

# Check if any token IDs are out of range
if input_ids.max() >= tokenizer_fresh.vocab_size:
    print(f"ERROR: Input token {input_ids.max()} >= vocab size {tokenizer_fresh.vocab_size}")

valid_labels = labels[labels != -100]
if len(valid_labels) > 0 and valid_labels.max() >= tokenizer_fresh.vocab_size:
    print(f"ERROR: Label token {valid_labels.max()} >= vocab size {tokenizer_fresh.vocab_size}")

# Check for any invalid values
print(f"Any NaN in input_ids: {torch.isnan(input_ids.float()).any()}")
print(f"Any NaN in labels: {torch.isnan(labels.float()).any()}")

# Decode the problematic tokens
print("\nDecoding batch tokens:")
for i in range(len(input_ids)):
    input_decoded = tokenizer_fresh.decode(input_ids[i])
    valid_label_tokens = [t for t in labels[i] if t != -100]
    if valid_label_tokens:
        label_decoded = tokenizer_fresh.decode(valid_label_tokens)
    else:
        label_decoded = "NO VALID LABELS"

    print(f"Sample {i}:")
    print(f"  Input: {input_decoded}")
    print(f"  Label: {label_decoded}")
    print(f"  Label tokens: {valid_label_tokens}")

Debugging NaN loss...
Model vocabulary size: 250112
Tokenizer vocabulary size: 250100
Checking token ID ranges...
Input IDs range: min=0, max=182922
Labels range: min=1, max=11947
Any NaN in input_ids: False
Any NaN in labels: False

Decoding batch tokens:
Sample 0:
  Input: classify: နည်းပညာ သတင်း</s>
  Label: Technology</s>
  Label tokens: [tensor(11947), tensor(1)]
Sample 1:
  Input: classify: အားကစား သတင်း</s><pad>
  Label: Sports</s>
  Label tokens: [tensor(11340), tensor(1)]


In [ ]:
# Fix vocabulary mismatch
print("Fixing vocabulary mismatch...")

print(f"Model vocab size: {model_fresh.config.vocab_size}")
print(f"Tokenizer vocab size: {tokenizer_fresh.vocab_size}")

# Fix the mismatch
model_fresh.config.vocab_size = tokenizer_fresh.vocab_size
model_fresh.resize_token_embeddings(tokenizer_fresh.vocab_size)

print("Vocabulary mismatch fixed!")

# Test the fix
simple_inputs = ["classify: test news"]
simple_targets = ["Technology"]

input_tokens = tokenizer_fresh.encode(simple_inputs[0], add_special_tokens=True)
label_tokens = tokenizer_fresh.encode(simple_targets[0], add_special_tokens=True)

test_batch = {
    'input_ids': torch.tensor([input_tokens]),
    'labels': torch.tensor([label_tokens])
}

if torch.cuda.is_available():
    test_batch = {k: v.cuda() for k, v in test_batch.items()}

model_fresh.train()
test_outputs = model_fresh(**test_batch)
test_loss = test_outputs.loss
print(f"Test loss after fix: {test_loss.item()}")

if not torch.isnan(test_loss) and test_loss.item() > 0:
    print("✓ SUCCESS: Fix worked!")
else:
    print(f"✗ Still issues: loss = {test_loss.item()}")

Fixing vocabulary mismatch...
Model vocab size: 250112
Tokenizer vocab size: 250100
Vocabulary mismatch fixed!
Test loss after fix: 20.5703125
✓ SUCCESS: Fix worked!


In [ ]:
# Complete model reload and proper fix
print("Reloading model completely to fix embedding issue...")

# Clear memory completely
import torch
import gc
torch.cuda.empty_cache()
gc.collect()

# Delete existing model
del model_fresh
del tokenizer_fresh
torch.cuda.empty_cache()

# Reload completely fresh model
print("Loading completely fresh model...")
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_name = "google/mt5-small"
tokenizer_new = T5Tokenizer.from_pretrained(model_name)
model_new = T5ForConditionalGeneration.from_pretrained(model_name)

print(f"Fresh model vocab size: {model_new.config.vocab_size}")
print(f"Fresh tokenizer vocab size: {tokenizer_new.vocab_size}")

# Check if mismatch exists
if model_new.config.vocab_size != tokenizer_new.vocab_size:
    print(f"Mismatch detected: {model_new.config.vocab_size} vs {tokenizer_new.vocab_size}")
    print("Properly resizing embeddings...")

    # Proper resize method
    model_new.resize_token_embeddings(tokenizer_new.vocab_size)

    # Verify the resize worked
    print(f"After resize - Model vocab: {model_new.config.vocab_size}")
    print(f"After resize - Shared embedding size: {model_new.shared.num_embeddings}")
    print(f"After resize - LM head size: {model_new.lm_head.out_features}")

    # Double check all sizes match
    assert model_new.config.vocab_size == tokenizer_new.vocab_size
    assert model_new.shared.num_embeddings == tokenizer_new.vocab_size
    assert model_new.lm_head.out_features == tokenizer_new.vocab_size

    print("✓ All embedding sizes properly synchronized")
else:
    print("✓ No mismatch - model ready")

# Move to GPU
if torch.cuda.is_available():
    model_new = model_new.cuda()

print("Model properly loaded and fixed")

Reloading model completely to fix embedding issue...
Loading completely fresh model...


You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


Fresh model vocab size: 250112
Fresh tokenizer vocab size: 250100
Mismatch detected: 250112 vs 250100
Properly resizing embeddings...
After resize - Model vocab: 250100
After resize - Shared embedding size: 250100
After resize - LM head size: 250100
✓ All embedding sizes properly synchronized
Model properly loaded and fixed


In [ ]:
print("Testing properly fixed model...")

# Create simple test data
test_inputs = ["classify: technology news"]
test_targets = ["Technology"]

# Tokenize
input_tokens = tokenizer_new.encode(test_inputs[0], add_special_tokens=True)
label_tokens = tokenizer_new.encode(test_targets[0], add_special_tokens=True)

print(f"Input tokens: {input_tokens}")
print(f"Label tokens: {label_tokens}")
print(f"Max input token: {max(input_tokens)}")
print(f"Max label token: {max(label_tokens)}")

# Verify token ranges
if max(input_tokens) < tokenizer_new.vocab_size and max(label_tokens) < tokenizer_new.vocab_size:
    print("✓ All tokens within range")

# Test forward pass
test_batch = {
    'input_ids': torch.tensor([input_tokens]),
    'labels': torch.tensor([label_tokens])
}

if torch.cuda.is_available():
    test_batch = {k: v.cuda() for k, v in test_batch.items()}

model_new.train()
try:
    test_outputs = model_new(**test_batch)
    test_loss = test_outputs.loss
    print(f"Forward pass loss: {test_loss.item()}")

    if not torch.isnan(test_loss) and test_loss.item() > 0:
        print("✓ Forward pass working correctly")

        # Test backward pass
        test_loss.backward()
        print("✓ Backward pass working correctly")

        print("Model is ready for training!")

    else:
        print(f"✗ Loss issue: {test_loss.item()}")

except Exception as e:
    print(f"Forward/backward pass failed: {e}")

Testing properly fixed model...
Input tokens: [3931, 30077, 267, 16185, 6781, 1]
Label tokens: [11947, 1]
Max input token: 30077
Max label token: 11947
✓ All tokens within range
Forward pass loss: 26.70096206665039
✓ Forward pass working correctly
✓ Backward pass working correctly
Model is ready for training!


In [ ]:
print("Training with properly fixed model...")

# Create training dataset
train_inputs = ["classify: technology news", "classify: sports news", "classify: business news"] * 10
train_targets = ["Technology", "Sports", "Business"] * 10

def tokenize_proper(examples):
    model_inputs = tokenizer_new(
        examples['input_text'],
        max_length=32,
        truncation=True,
        padding=False
    )

    with tokenizer_new.as_target_tokenizer():
        labels = tokenizer_new(
            examples['target_text'],
            max_length=8,
            truncation=True,
            padding=False
        )

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

proper_dataset = Dataset.from_dict({
    'input_text': train_inputs,
    'target_text': train_targets
}).map(tokenize_proper, batched=True).remove_columns(['input_text', 'target_text'])

print(f"Dataset created: {len(proper_dataset)} samples")

# Training arguments
training_args_proper = TrainingArguments(
    output_dir="./properly-fixed",
    max_steps=20,
    per_device_train_batch_size=2,
    learning_rate=5e-5,
    logging_steps=5,
    save_strategy="no",
    eval_strategy="no",
    push_to_hub=False,
    report_to=None,
    fp16=False,
)

# Create trainer
trainer_proper = Trainer(
    model=model_new,
    args=training_args_proper,
    train_dataset=proper_dataset,
    data_collator=DataCollatorForSeq2Seq(tokenizer_new, model_new),
)

print("Starting properly fixed training...")
try:
    result_proper = trainer_proper.train()
    print(f"Training completed!")
    print(f"Final loss: {result_proper.training_loss}")

    if result_proper.training_loss > 0:
        print("SUCCESS: Training finally works!")

        # Test the trained model
        print("Testing trained model...")
        test_input = "classify: technology news"
        inputs = tokenizer_new(test_input, return_tensors="pt")
        if torch.cuda.is_available():
            inputs = {k: v.cuda() for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model_new.generate(**inputs, max_length=20)

        result = tokenizer_new.decode(outputs[0], skip_special_tokens=True)
        print(f"Model prediction: {result}")

        if "Technology" in result:
            print("PERFECT: Model learned classification!")
        else:
            print("Model trained successfully (output needs refinement)")

    else:
        print("Still getting 0 loss")

except Exception as e:
    print(f"Training failed: {e}")

Training with properly fixed model...


Map:   0%|          | 0/30 [00:00<?, ? examples/s]

Dataset created: 30 samples
Starting properly fixed training...


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3951: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Step,Training Loss
5,0.000000
10,0.000000
15,0.000000
20,0.000000


Training completed!
Final loss: 0.0
Still getting 0 loss


In [ ]:
# Debug what happens during training
print("Debugging the training loop...")

# Let's manually step through one training iteration
model_new.train()

# Get one batch manually
train_batch = [proper_dataset[i] for i in range(2)]
collated_batch = DataCollatorForSeq2Seq(tokenizer_new, model_new)(train_batch)

if torch.cuda.is_available():
    batch_cuda = {k: v.cuda() if torch.is_tensor(v) else v for k, v in collated_batch.items()}
else:
    batch_cuda = collated_batch

print("Manual training step...")
print(f"Batch input_ids shape: {batch_cuda['input_ids'].shape}")
print(f"Batch labels shape: {batch_cuda['labels'].shape}")

# Manual forward pass
outputs = model_new(**batch_cuda)
loss = outputs.loss

print(f"Manual training loss: {loss.item()}")
print(f"Loss is valid: {not torch.isnan(loss) and loss.item() > 0}")

# Check if loss requires grad
print(f"Loss requires grad: {loss.requires_grad}")

# Manual backward pass
loss.backward()
print("Manual backward pass completed")

# Check gradients
has_grads = any(p.grad is not None for p in model_new.parameters())
print(f"Model has gradients: {has_grads}")

if has_grads:
    # Check gradient norms
    total_norm = torch.nn.utils.clip_grad_norm_(model_new.parameters(), float('inf'))
    print(f"Gradient norm: {total_norm}")
else:
    print("No gradients found!")

Debugging the training loop...
Manual training step...
Batch input_ids shape: torch.Size([2, 6])
Batch labels shape: torch.Size([2, 2])
Manual training loss: nan
Loss is valid: False
Loss requires grad: True
Manual backward pass completed
Model has gradients: True
Gradient norm: nan


In [ ]:
# Try different training setup
print("Trying different training approach...")

# Clear gradients
model_new.zero_grad()

# Try with a simple training loop instead of Trainer
print("Manual training loop...")

# Simple optimizer
from torch.optim import AdamW
optimizer = AdamW(model_new.parameters(), lr=5e-5)

# Manual training for a few steps
losses = []

for step in range(5):
    # Get batch
    start_idx = (step * 2) % len(proper_dataset)
    batch_indices = [start_idx, (start_idx + 1) % len(proper_dataset)]

    train_batch = [proper_dataset[i] for i in batch_indices]
    collated = DataCollatorForSeq2Seq(tokenizer_new, model_new)(train_batch)

    if torch.cuda.is_available():
        batch = {k: v.cuda() if torch.is_tensor(v) else v for k, v in collated.items()}
    else:
        batch = collated

    # Forward pass
    model_new.train()
    outputs = model_new(**batch)
    loss = outputs.loss

    print(f"Step {step+1}: loss = {loss.item():.6f}")
    losses.append(loss.item())

    # Backward pass
    loss.backward()

    # Optimizer step
    optimizer.step()
    optimizer.zero_grad()

print(f"Manual training losses: {losses}")

if all(l > 0 for l in losses):
    print("✓ Manual training works! Issue is with Trainer class")

    # Test the manually trained model
    print("Testing manually trained model...")
    test_input = "classify: technology news"
    inputs = tokenizer_new(test_input, return_tensors="pt")
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}

    model_new.eval()
    with torch.no_grad():
        outputs = model_new.generate(**inputs, max_length=20)

    result = tokenizer_new.decode(outputs[0], skip_special_tokens=True)
    print(f"Manually trained model output: {result}")

else:
    print("Manual training also fails")

Trying different training approach...
Manual training loop...
Step 1: loss = nan
Step 2: loss = nan
Step 3: loss = nan
Step 4: loss = nan
Step 5: loss = nan
Manual training losses: [nan, nan, nan, nan, nan]
Manual training also fails


In [ ]:
# Try alternative Trainer setup
print("Trying alternative Trainer configuration...")

# Sometimes the issue is with training arguments
# Let's try more basic settings

training_args_alt = TrainingArguments(
    output_dir="./alternative",
    num_train_epochs=1,  # Use epochs instead of max_steps
    per_device_train_batch_size=1,  # Batch size 1
    learning_rate=1e-4,  # Higher learning rate
    logging_steps=1,
    save_strategy="no",
    eval_strategy="no",
    push_to_hub=False,
    report_to=None,
    fp16=False,
    dataloader_num_workers=0,
    remove_unused_columns=False,  # Don't remove columns
    dataloader_pin_memory=False,   # Disable pin memory
)

# Simpler data collator
class SimpleCollator:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, features):
        # Simple padding
        max_input_len = max(len(f['input_ids']) for f in features)
        max_label_len = max(len(f['labels']) for f in features)

        input_ids = []
        labels = []
        attention_mask = []

        for f in features:
            # Pad input_ids
            input_id = f['input_ids'] + [self.tokenizer.pad_token_id] * (max_input_len - len(f['input_ids']))
            input_ids.append(input_id)

            # Attention mask
            mask = [1] * len(f['input_ids']) + [0] * (max_input_len - len(f['input_ids']))
            attention_mask.append(mask)

            # Pad labels
            label = f['labels'] + [-100] * (max_label_len - len(f['labels']))
            labels.append(label)

        return {
            'input_ids': torch.tensor(input_ids),
            'attention_mask': torch.tensor(attention_mask),
            'labels': torch.tensor(labels)
        }

simple_collator = SimpleCollator(tokenizer_new)

# Create alternative trainer
trainer_alt = Trainer(
    model=model_new,
    args=training_args_alt,
    train_dataset=proper_dataset.select(range(10)),  # Only 10 samples
    data_collator=simple_collator,
)

print("Starting alternative trainer...")
try:
    result_alt = trainer_alt.train()
    print(f"Alternative trainer result: {result_alt.training_loss}")

    if result_alt.training_loss > 0:
        print("🎉 Alternative trainer works!")
    else:
        print("Alternative trainer also gives 0 loss")

except Exception as e:
    print(f"Alternative trainer failed: {e}")

Trying alternative Trainer configuration...
Starting alternative trainer...


Step,Training Loss
1,0.000000
2,0.000000
3,0.000000
4,0.000000
5,0.000000
6,0.000000
7,0.000000
8,0.000000
9,0.000000
10,0.000000


Alternative trainer result: 0.0
Alternative trainer also gives 0 loss


In [ ]:
# Debug data collator corruption
print("Debugging data collator...")

# Test single sample (this worked)
print("=== SINGLE SAMPLE TEST ===")
single_sample = proper_dataset[0]
print(f"Single sample input_ids: {single_sample['input_ids']}")
print(f"Single sample labels: {single_sample['labels']}")

# Manually create single batch
single_batch = {
    'input_ids': torch.tensor([single_sample['input_ids']]),
    'labels': torch.tensor([single_sample['labels']])
}

if torch.cuda.is_available():
    single_batch = {k: v.cuda() for k, v in single_batch.items()}

# Test single batch
model_new.eval()
with torch.no_grad():
    single_output = model_new(**single_batch)
    single_loss = single_output.loss

print(f"Single batch loss: {single_loss.item()}")

# Now test data collator with same sample
print("\n=== DATA COLLATOR TEST ===")
collator = DataCollatorForSeq2Seq(tokenizer_new, model_new)

# Collate single sample
collated_single = collator([single_sample])
print(f"Collated single - input_ids shape: {collated_single['input_ids'].shape}")
print(f"Collated single - labels shape: {collated_single['labels'].shape}")
print(f"Collated single - input_ids: {collated_single['input_ids'][0]}")
print(f"Collated single - labels: {collated_single['labels'][0]}")

if torch.cuda.is_available():
    collated_single = {k: v.cuda() if torch.is_tensor(v) else v for k, v in collated_single.items()}

# Test collated single
with torch.no_grad():
    collated_single_output = model_new(**collated_single)
    collated_single_loss = collated_single_output.loss

print(f"Collated single loss: {collated_single_loss.item()}")

# Now test with two samples
print("\n=== TWO SAMPLES TEST ===")
two_samples = [proper_dataset[0], proper_dataset[1]]
print(f"Sample 0 input_ids: {two_samples[0]['input_ids']}")
print(f"Sample 0 labels: {two_samples[0]['labels']}")
print(f"Sample 1 input_ids: {two_samples[1]['input_ids']}")
print(f"Sample 1 labels: {two_samples[1]['labels']}")

# Collate two samples
collated_two = collator(two_samples)
print(f"Collated two - input_ids shape: {collated_two['input_ids'].shape}")
print(f"Collated two - labels shape: {collated_two['labels'].shape}")
print(f"Collated two - input_ids[0]: {collated_two['input_ids'][0]}")
print(f"Collated two - input_ids[1]: {collated_two['input_ids'][1]}")
print(f"Collated two - labels[0]: {collated_two['labels'][0]}")
print(f"Collated two - labels[1]: {collated_two['labels'][1]}")

# Check for issues in collated data
print("\n=== CHECKING FOR ISSUES ===")
print(f"Any NaN in input_ids: {torch.isnan(collated_two['input_ids'].float()).any()}")
print(f"Any NaN in labels: {torch.isnan(collated_two['labels'].float()).any()}")
print(f"Input_ids range: {collated_two['input_ids'].min()} to {collated_two['input_ids'].max()}")

# Check labels (exclude -100 padding)
valid_labels = collated_two['labels'][collated_two['labels'] != -100]
if len(valid_labels) > 0:
    print(f"Valid labels range: {valid_labels.min()} to {valid_labels.max()}")
    print(f"Valid labels: {valid_labels}")
else:
    print("NO VALID LABELS FOUND!")

# Test two samples
if torch.cuda.is_available():
    collated_two = {k: v.cuda() if torch.is_tensor(v) else v for k, v in collated_two.items()}

with torch.no_grad():
    collated_two_output = model_new(**collated_two)
    collated_two_loss = collated_two_output.loss

print(f"Collated two samples loss: {collated_two_loss.item()}")

Debugging data collator...
=== SINGLE SAMPLE TEST ===
Single sample input_ids: [3931, 30077, 267, 16185, 6781, 1]
Single sample labels: [11947, 1]
Single batch loss: nan

=== DATA COLLATOR TEST ===
Collated single - input_ids shape: torch.Size([1, 6])
Collated single - labels shape: torch.Size([1, 2])
Collated single - input_ids: tensor([ 3931, 30077,   267, 16185,  6781,     1])
Collated single - labels: tensor([11947,     1])
Collated single loss: nan

=== TWO SAMPLES TEST ===
Sample 0 input_ids: [3931, 30077, 267, 16185, 6781, 1]
Sample 0 labels: [11947, 1]
Sample 1 input_ids: [3931, 30077, 267, 21318, 6781, 1]
Sample 1 labels: [11340, 1]
Collated two - input_ids shape: torch.Size([2, 6])
Collated two - labels shape: torch.Size([2, 2])
Collated two - input_ids[0]: tensor([ 3931, 30077,   267, 16185,  6781,     1])
Collated two - input_ids[1]: tensor([ 3931, 30077,   267, 21318,  6781,     1])
Collated two - labels[0]: tensor([11947,     1])
Collated two - labels[1]: tensor([11340,  

In [ ]:
# Create fixed data collator
print("Creating fixed data collator...")

class FixedDataCollator:
    def __init__(self, tokenizer, pad_to_multiple_of=None):
        self.tokenizer = tokenizer
        self.pad_to_multiple_of = pad_to_multiple_of

    def __call__(self, features):
        # Get max lengths
        max_input_length = max(len(f["input_ids"]) for f in features)
        max_label_length = max(len(f["labels"]) for f in features)

        # Prepare batch
        batch = {}

        # Pad input_ids and create attention_mask
        input_ids = []
        attention_mask = []

        for f in features:
            input_id = f["input_ids"]

            # Calculate padding needed
            padding_length = max_input_length - len(input_id)

            # Pad input_ids
            if self.tokenizer.padding_side == "right":
                padded_input = input_id + [self.tokenizer.pad_token_id] * padding_length
                mask = [1] * len(input_id) + [0] * padding_length
            else:
                padded_input = [self.tokenizer.pad_token_id] * padding_length + input_id
                mask = [0] * padding_length + [1] * len(input_id)

            input_ids.append(padded_input)
            attention_mask.append(mask)

        batch["input_ids"] = torch.tensor(input_ids, dtype=torch.long)
        batch["attention_mask"] = torch.tensor(attention_mask, dtype=torch.long)

        # Pad labels
        labels = []
        for f in features:
            label = f["labels"]
            padding_length = max_label_length - len(label)

            # Pad labels with -100 (ignore index)
            if self.tokenizer.padding_side == "right":
                padded_label = label + [-100] * padding_length
            else:
                padded_label = [-100] * padding_length + label

            labels.append(padded_label)

        batch["labels"] = torch.tensor(labels, dtype=torch.long)

        return batch

# Test fixed collator
fixed_collator = FixedDataCollator(tokenizer_new)

print("Testing fixed collator...")

# Test with single sample
single_test = fixed_collator([proper_dataset[0]])
print(f"Fixed single - input shape: {single_test['input_ids'].shape}")
print(f"Fixed single - labels shape: {single_test['labels'].shape}")

if torch.cuda.is_available():
    single_test = {k: v.cuda() for k, v in single_test.items()}

with torch.no_grad():
    single_test_output = model_new(**single_test)
    print(f"Fixed single loss: {single_test_output.loss.item()}")

# Test with two samples
two_test = fixed_collator([proper_dataset[0], proper_dataset[1]])
print(f"Fixed two - input shape: {two_test['input_ids'].shape}")
print(f"Fixed two - labels shape: {two_test['labels'].shape}")

if torch.cuda.is_available():
    two_test = {k: v.cuda() for k, v in two_test.items()}

with torch.no_grad():
    two_test_output = model_new(**two_test)
    print(f"Fixed two samples loss: {two_test_output.loss.item()}")

if not torch.isnan(two_test_output.loss) and two_test_output.loss.item() > 0:
    print("Fixed collator works!")
else:
    print(f"Fixed collator still has issues: {two_test_output.loss.item()}")

Creating fixed data collator...
Testing fixed collator...
Fixed single - input shape: torch.Size([1, 6])
Fixed single - labels shape: torch.Size([1, 2])
Fixed single loss: nan
Fixed two - input shape: torch.Size([2, 6])
Fixed two - labels shape: torch.Size([2, 2])
Fixed two samples loss: nan
Fixed collator still has issues: nan
